In [9]:
import os
import re
import numpy as np
from datetime import datetime

In [12]:
def analyze_logs(filepath):
    clock_regex = re.compile(r"(\d+)\)")
    time_regex = re.compile(r'^(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3})')

    clock_jumps = []
    time_jumps = []
    experiment_clock_rates = []
    with open(filepath, 'r') as file:
        data = file.readlines()
        # get the clock rate from the first line
        clock_rate = int(data[0][-2:])
        # skip the first line that prints out the clock rate
        last_time = None
        last_clock = None
        for line in data[1:]:
            match = clock_regex.search(line)
            logical_clock = int(match.group(1))
            match = time_regex.search(line)
            time_stamp = datetime.strptime(match.group(1), '%Y-%m-%d %H:%M:%S,%f')

            if last_time and last_clock:
                time_diff = (time_stamp - last_time).microseconds / 1000000
                clock_diff = logical_clock - last_clock
                clock_jumps.append(clock_diff)
                time_jumps.append(time_diff)
                experiment_clock_rates.append(time_diff/clock_diff)

            last_clock = logical_clock
            last_time = time_stamp
            
    # print format string but only up to 2 decimal places
    print("Actual clock rate: {:.3f} (tick every {:.3f} seconds)".format(clock_rate, 1/clock_rate))
    print("\t\t\t\t\t\t\t Avg \t\t Max \t\t  Min")
    print("Difference in logical clock between machine operations:\t {:.3f} \t\t {:.3f} \t\t {:.3f}".format(np.mean(clock_jumps), np.max(clock_jumps), np.min(clock_jumps)))
    print("Difference in system time between machine operations: \t {:.3f} \t\t {:.3f} \t\t {:.3f}".format(np.mean(time_jumps), np.max(time_jumps), np.min(time_jumps)))
    print("Experiment clock rates: (seconds between clock ticks): \t {:.3f} \t\t {:.3f} \t\t {:.3f}\n".format(np.mean(experiment_clock_rates), np.max(experiment_clock_rates), np.min(experiment_clock_rates)))




In [23]:
log_folders = sorted(os.listdir("logs"))
for experiment_folder in range(len(log_folders)):
    print("## Experiment {}:".format(experiment_folder + 1))
    directory = os.path.join("logs", log_folders[experiment_folder])
    files = sorted(os.listdir(directory))
    for file_index in range(len(files)):
        f = os.path.join(directory, files[file_index])
        print("### Machine {}:".format(file_index))
        analyze_logs(f)

Experiment 1:
Machine 0:
Actual clock rate: 1.000 (tick every 1.000 seconds)
							 Avg 		 Max 		  Min
Difference in logical clock between machine operations:	 3.869 		 16.000 		 1.000
Difference in system time between machine operations: 	 0.003 		 0.008 		 0.000
Experiment clock rates: (seconds between clock ticks): 	 0.002 		 0.006 		 0.000

Machine 1:
Actual clock rate: 3.000 (tick every 0.333 seconds)
							 Avg 		 Max 		  Min
Difference in logical clock between machine operations:	 1.953 		 15.000 		 1.000
Difference in system time between machine operations: 	 0.337 		 0.349 		 0.333
Experiment clock rates: (seconds between clock ticks): 	 0.268 		 0.339 		 0.023

Machine 2:
Actual clock rate: 6.000 (tick every 0.167 seconds)
							 Avg 		 Max 		  Min
Difference in logical clock between machine operations:	 1.000 		 1.000 		 1.000
Difference in system time between machine operations: 	 0.170 		 0.181 		 0.166
Experiment clock rates: (seconds between clock ticks): 	 0.170 		 0